In [1]:
%load_ext watermark

import cf_xarray  # to show off
import rioxarray
import xarray as xr  # need >= 2022.10.0
from xarray.core.geoindex import with_geoindex

%watermark -iv


ds = xr.open_dataset(
    #"/Users/deepak/repos/rioxarray/test/test_data/input/PLANET_SCOPE_3D.nc",
    "./S_20240101_concentration_v3.0.tif",
    # decode_coords="all",
    engine="rasterio",
    backend_kwargs={"parse_coordinates": False},
)
ds

cf_xarray: 0.10.0
xarray   : 2025.1.2.dev18+g59383091
rioxarray: 0.18.2



<xarray.Dataset> Size: 420kB
Dimensions:      (band: 1, y: 332, x: 316)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
Dimensions without coordinates: y, x
Data variables:
    band_data    (band, y, x) float32 420kB ...

In [2]:
newds = with_geoindex(ds)
newds

<xarray.Dataset> Size: 2MB
Dimensions:      (band: 1, y: 332, x: 316)
Coordinates:
  * band         (band) int64 8B 1
  * spatial_ref  int64 8B 0
  * xc           (y, x) float64 839kB -3.938e+06 -3.912e+06 ... 3.938e+06
  * yc           (y, x) float64 839kB 4.338e+06 4.338e+06 ... -3.938e+06
Dimensions without coordinates: y, x
Data variables:
    band_data    (band, y, x) float32 420kB ...
Indexes:
  ┌ spatial_ref  GeoTransformIndex
  │ xc
  └ yc

In [5]:
# FIXME: 3 GeoTransformIndex objects now!
newds.isel(x=slice(None, None, 2), y=slice(None, None, 2))

<xarray.Dataset> Size: 525kB
Dimensions:      (band: 1, y: 166, x: 158)
Coordinates:
  * band         (band) int64 8B 1
  * spatial_ref  int64 8B 0
    xc           (y, x) float64 210kB -3.938e+06 -3.888e+06 ... 3.912e+06
    yc           (y, x) float64 210kB 4.338e+06 4.338e+06 ... -3.912e+06
Dimensions without coordinates: y, x
Data variables:
    band_data    (band, y, x) float32 105kB ...
Indexes:
    spatial_ref  GeoTransformIndex

In [6]:
# broken after I added isel
newds.sel(xc=[120, 121], yc=[-88.5, -88], options=dict(crs="EPSG:4326"))

AssertionError: 

In [4]:
# Before I added isel
newds.sel(xc=[120, 121], yc=[-88.5, -88], options=dict(crs="EPSG:4326"))

<xarray.Dataset> Size: 96B
Dimensions:      (band: 1, xc: 2, yc: 2)
Coordinates:
  * band         (band) int64 8B 1
  * spatial_ref  int64 8B 0
    xc           (xc, yc) float64 32B 1.375e+05 1.375e+05 1.875e+05 1.875e+05
    yc           (xc, yc) float64 32B -8.75e+04 -1.125e+05 -8.75e+04 -1.125e+05
Data variables:
    band_data    (band, xc, yc) float32 16B ...
Indexes:
    spatial_ref  GeoTransformIndex